In [1]:
#This is a praction to test CAPM.Please note that in reality we should use portfolio,instead of individual stocks,to do the test. 

In [2]:
import myfunc as mf
import wrds
db = wrds.Connection(wrds_username='fxw174')
import pandas as pd
from scipy import stats as st
import matplotlib.pyplot as plt
import numpy as np
import math

Loading library list...
Done
Loading library list...
Done


In [3]:
#This are functions writeen by me.Some parameters may not be used in this task,since I decide to write them as general as possible at first place.

#This function let you combine local DataFrame with the ones in the WRDS.
def up_sql(query,df):
    col=df.columns
    col=','.join(col)
    df=list(df.itertuples(index=False,name=None))
    add_query=" (VALUES """+("%s,"*len(df))[0:-1]+") AS u ("+col+")"
    query=query.format(add_query)
    return db.raw_sql(query,params=[df])

#convert cusip to gvkey
def cusip_gvkey(df=0,cusip_digit=8):    
    link=db.raw_sql("select distinct cusip,gvkey from comp.funda")
    if cusip_digit==8:
        def cusip9to8(cusip9):
            if cusip9:
                return cusip9[0:-1]
        link['cusip']=link['cusip'].apply(cusip9to8)
    if type(df)==int:
        return link
    else:
        return link.merge(df,on='cusip')

#get beta
def get_beta(df='',estimate=False):
    if type(df)==str:
        if df:
            date=pd.to_datetime(df)
        else:
            date=pd.to_datetime('now')-pd.DateOffset(months=1)
        year=date.year
        month=date.month
        df=db.raw_sql("""
        SELECT
            gvkey,
            beta,
            datadate as date
        FROM
            comp_na_daily_all.co_mthly
        WHERE
            %(year)s=EXTRACT(YEAR FROM datadate) AND
            %(month)s=EXTRACT(MONTH FROM datadate);
        """,params={'year':year,'month':month},date_cols=['date'])
    else:
        if 'date' not in df.columns:
            df['date']=pd.to_datetime('now')-pd.DateOffset(months=1)
        if 'gvkey' not in df.columns:
            df=cusip_gvkey(df)
        df1=df.drop_duplicates(['gvkey','date'])[['gvkey','date']]
        query="""
        SELECT  
            u.*,
            a.beta
        FROM
            comp_na_daily_all.co_mthly AS a,{}
        WHERE
            u.gvkey=a.gvkey AND
            EXTRACT(YEAR FROM u.date)=EXTRACT(YEAR FROM a.datadate) AND
            EXTRACT(MONTH FROM u.date)=EXTRACT(MONTH FROM a.datadate);
        """
        df1=up_sql(query,df1)
        df=df.merge(df1,on=['gvkey','date'])
    if estimate:
        from scipy.stats.mstats import gmean
        factors=db.raw_sql("SELECT date,rf,mktrf FROM ff.factors_monthly",date_cols=['date'])
        factors['year']=pd.DatetimeIndex(factors['date']).year
        factors['month']=pd.DatetimeIndex(factors['date']).month
        
        def estimate_beta(df):
            date=df.name
            result=factors[(factors['date']<=date)&(factors['date']>=date-pd.DateOffset(years=20))]['mktrf']+1
            result=gmean(result)
            df['mktrf_20yr_mean']=result-1
            return df
        df=df.groupby('date').apply(estimate_beta)
        df['year']=pd.DatetimeIndex(df['date']).year
        df['month']=pd.DatetimeIndex(df['date']).month
        df=df.merge(factors[['rf','year','month']],on=['year','month'])
        df['est_CAPM']=df['rf']+df['beta']*df['mktrf_20yr_mean']
        df=df.drop(['month','year','mktrf_20yr_mean','rf'],axis=1)
    return df

In [4]:
df=db.raw_sql("""
SELECT
    cusip
    ,date
    ,ret
FROM
    crsp.msf
""",date_cols=['date'])

In [5]:
df['ret']=(df['ret']+1).apply(math.log) #get the log return
df=df.groupby(['cusip',pd.Grouper(key='date',freq='Y')]).aggregate(['sum','count']).reset_index() #get the yearly return
df.columns=['cusip','ret_date','ret','count']
df=df[df['count']==12] # only keep the valid return
df['date']=df['ret_date']-pd.DateOffset(years=1) # lag
df=get_beta(df)# get beta
df=df.dropna()
st.linregress(df['beta'],df['ret']) # Run regression.The result shows the CAPM has limited prediction power. 

LinregressResult(slope=-6.146546152624917e-05, intercept=0.007730277861168674, rvalue=-0.006455336479267456, pvalue=0.020847137519136104, stderr=2.659955744106956e-05, intercept_stderr=0.0015273533894620403)